In [2]:
from __future__ import division, print_function,unicode_literals

import numpy as np
import os 

np.random.seed(42)
%matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes',labelsize=14)
mpl.rc('xtick',labelsize=12)
mpl.rc('ytick',labelsize=12)

Using matplotlib backend: Qt5Agg
